# Механизми пажње и трансформери

Један од главних недостатака рекурентних мрежа је то што све речи у низу имају исти утицај на резултат. Ово доводи до субоптималних перформанси код стандардних LSTM енкодер-декодер модела за задатке претварања секвенци, као што су препознавање именованих ентитета и машински превод. У стварности, одређене речи у улазној секвенци често имају већи утицај на излазне секвенце од других.

Размотримо модел претварања секвенци, као што је машински превод. Он се имплементира помоћу две рекурентне мреже, где једна мрежа (**енкодер**) компримује улазну секвенцу у скривено стање, а друга, **декодер**, развија ово скривено стање у преведени резултат. Проблем са овим приступом је што завршно стање мреже тешко памти почетак реченице, што доводи до лошег квалитета модела код дугих реченица.

**Механизми пажње** пружају начин да се тежински одреди контекстуални утицај сваког улазног вектора на сваку излазну предикцију RNN-а. Ово се имплементира стварањем пречица између интермедијарних стања улазног RNN-а и излазног RNN-а. На овај начин, када генеришемо излазни симбол $y_t$, узимамо у обзир сва улазна скривена стања $h_i$, са различитим тежинским коефицијентима $\alpha_{t,i}$.

![Слика која приказује енкодер/декодер модел са адитивним слојем пажње](../../../../../translated_images/sr/encoder-decoder-attention.7a726296894fb567.webp)  
*Енкодер-декодер модел са адитивним механизмом пажње у [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитиран из [овог блога](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матрица пажње $\{\alpha_{i,j}\}$ представља степен у којем одређене улазне речи утичу на генерисање одређене речи у излазној секвенци. Испод је пример такве матрице:

![Слика која приказује пример поравнања пронађеног помоћу RNNsearch-50, преузета из Bahdanau - arviz.org](../../../../../translated_images/sr/bahdanau-fig3.09ba2d37f202a6af.webp)  

*Слика преузета из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Сл.3)*

Механизми пажње су одговорни за велики део тренутног или скоро тренутног стања уметности у обради природног језика. Међутим, додавање пажње значајно повећава број параметара модела, што је довело до проблема са скалирањем код RNN-ова. Кључно ограничење скалирања RNN-ова је то што рекурентна природа модела отежава груписање и паралелизацију тренинга. У RNN-у сваки елемент секвенце мора бити обрађен редом, што значи да се не може лако паралелизовати.

Усвајање механизама пажње у комбинацији са овим ограничењем довело је до стварања садашњих трансформер модела, који представљају стање уметности, а које данас познајемо и користимо, од BERT-а до OpenGPT3.

## Трансформер модели

Уместо да преносе контекст сваке претходне предикције у следећи корак евалуације, **трансформер модели** користе **позиционе кодирања** и пажњу како би ухватили контекст датог улаза у оквиру одређеног прозора текста. Слика испод показује како позициона кодирања са пажњом могу ухватити контекст унутар датог прозора.

![Анимирани GIF који приказује како се евалуације изводе у трансформер моделима.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Пошто се свака улазна позиција независно мапира на сваку излазну позицију, трансформери могу боље паралелизовати од RNN-ова, што омогућава много веће и изражајније језичке моделе. Свака глава пажње може се користити за учење различитих односа између речи, што побољшава задатке обраде природног језика.

**BERT** (Bidirectional Encoder Representations from Transformers) је веома велика трансформер мрежа са више слојева, са 12 слојева за *BERT-base*, и 24 за *BERT-large*. Модел се прво претходно тренира на великом корпусу текстуалних података (Википедија + књиге) користећи несупервизирано учење (предвиђање маскираних речи у реченици). Током претходног тренинга, модел апсорбује значајан ниво разумевања језика, који се затим може искористити са другим скуповима података кроз фино подешавање. Овај процес се назива **трансфер учење**.

![Слика са http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sr/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Постоји много варијација трансформер архитектура, укључујући BERT, DistilBERT, BigBird, OpenGPT3 и друге, које се могу фино подесити. [HuggingFace пакет](https://github.com/huggingface/) пружа репозиторијум за тренирање многих од ових архитектура са PyTorch-ом.

## Коришћење BERT-а за класификацију текста

Хајде да видимо како можемо користити претходно тренирани BERT модел за решавање нашег традиционалног задатка: класификације секвенци. Класификоваћемо наш оригинални AG News скуп података.

Прво, учитајмо HuggingFace библиотеку и наш скуп података:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Пошто ћемо користити унапред обучени BERT модел, потребно је да користимо одређени токенизатор. Прво ћемо учитати токенизатор који је повезан са унапред обученим BERT моделом.

Библиотека HuggingFace садржи репозиторијум унапред обучених модела, које можете користити једноставним навођењем њихових имена као аргумената функцији `from_pretrained`. Сви потребни бинарни фајлови за модел биће аутоматски преузети.

Међутим, у одређеним ситуацијама биће потребно да учитате сопствене моделе. У том случају можете навести директоријум који садржи све релевантне фајлове, укључујући параметре за токенизатор, `config.json` фајл са параметрима модела, бинарне тежине, итд.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` објекат садржи `encode` функцију која се може директно користити за кодирање текста:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Затим, хајде да направимо итераторе које ћемо користити током тренинга за приступ подацима. Пошто BERT користи своју функцију за енкодовање, морали бисмо да дефинишемо функцију за попуњавање сличну `padify` коју смо раније дефинисали:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

У нашем случају, користићемо претходно обучени BERT модел назван `bert-base-uncased`. Хајде да учитамо модел користећи пакет `BertForSequenceClassfication`. Ово осигурава да наш модел већ има потребну архитектуру за класификацију, укључујући завршни класификатор. Видећете поруку упозорења која наводи да тежине завршног класификатора нису иницијализоване и да модел захтева претходну обуку - то је потпуно у реду, јер је то управо оно што ћемо урадити!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Сада смо спремни да започнемо обуку! Пошто је BERT већ претходно обучен, желимо да започнемо са прилично малом стопом учења како не бисмо уништили почетне тежине.

Сав тежак посао обавља модел `BertForSequenceClassification`. Када позовемо модел на подацима за обуку, он враћа и губитак (loss) и излаз мреже за улазни мини-бач. Губитак користимо за оптимизацију параметара (`loss.backward()` врши повратни пролаз), а `out` користимо за израчунавање тачности обуке поређењем добијених ознака `labs` (израчунатих помоћу `argmax`) са очекиваним `labels`.

Да бисмо контролисали процес, акумулирамо губитак и тачност током неколико итерација и штампамо их сваки `report_freq` циклус обуке.

Ова обука ће вероватно трајати прилично дуго, па ограничавамо број итерација.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Можете видети (посебно ако повећате број итерација и сачекате довољно дуго) да класификација помоћу BERT-а даје прилично добру тачност! То је зато што BERT већ прилично добро разуме структуру језика, па је потребно само дорађивање завршног класификатора. Међутим, пошто је BERT велики модел, цео процес обуке траје дуго и захтева озбиљну рачунарску снагу! (GPU, и пожељно више од једног).

> **Note:** У нашем примеру користимо један од најмањих унапред обучених BERT модела. Постоје већи модели који вероватно дају боље резултате.


## Оцењивање перформанси модела

Сада можемо проценити перформансе нашег модела на тестном скупу података. Петља за оцењивање је прилично слична петљи за тренирање, али не смемо заборавити да пребацимо модел у режим оцењивања позивом `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Закључак

У овој јединици смо видели колико је лако узети унапред обучен језички модел из библиотеке **transformers** и прилагодити га нашем задатку класификације текста. Слично томе, BERT модели могу се користити за извлачење ентитета, одговарање на питања и друге NLP задатке.

Модели трансформера представљају тренутно најсавременије решење у области NLP-а, и у већини случајева треба да буду прво решење са којим ћете почети експериментисање приликом имплементације прилагођених NLP решења. Међутим, разумевање основних принципа рекурентних неуронских мрежа, о којима је било речи у овом модулу, изузетно је важно ако желите да изградите напредне неуронске моделе.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не сносимо одговорност за било каква неспоразумевања или погрешна тумачења која могу произаћи из коришћења овог превода.
